In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

# Download Market Data

In [2]:
start_date = "2010-01-01"

sp500 = yf.download("^GSPC", start=start_date)
vix = yf.download("^VIX", start=start_date)

sp500 = sp500[["Close"]].rename(columns={"Close": "SP500_Close"})
vix = vix[["Close"]].rename(columns={"Close": "VIX_Close"})

df = sp500.join(vix, how="inner")

df.head()

Failed to get ticker '^GSPC' reason: Failed to perform, curl: (6) Could not resolve host: query1.finance.yahoo.com. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price,SP500_Close,VIX_Close
Ticker,^GSPC,^VIX
Date,,
2010-01-04,1132.989990,20.040001
2010-01-05,1136.520020,19.350000
2010-01-06,1137.140015,19.160000
2010-01-07,1141.689941,19.059999
2010-01-08,1144.979980,18.129999


In [3]:
df.tail()

Price,SP500_Close,VIX_Close
Ticker,^GSPC,^VIX
Date,,
2026-02-13,6836.169922,20.600000
2026-02-17,6843.220215,20.290001
2026-02-18,6881.310059,19.620001
2026-02-19,6861.890137,20.230000
2026-02-20,6909.509766,19.090000


# Compute Log Returns

In [4]:
df["SP500_LogReturn"] = np.log(
    df["SP500_Close"] / df["SP500_Close"].shift(1)
)

df.head()

Price,SP500_Close,VIX_Close,SP500_LogReturn
Ticker,^GSPC,^VIX,
Date,,,
2010-01-04,1132.989990,20.040001,NaN
2010-01-05,1136.520020,19.350000,0.003111
2010-01-06,1137.140015,19.160000,0.000545
2010-01-07,1141.689941,19.059999,0.003993
2010-01-08,1144.979980,18.129999,0.002878


In [5]:
df = df.dropna()

In [6]:
df.head()

Price,SP500_Close,VIX_Close,SP500_LogReturn
Ticker,^GSPC,^VIX,
Date,,,
2010-01-05,1136.520020,19.350000,0.003111
2010-01-06,1137.140015,19.160000,0.000545
2010-01-07,1141.689941,19.059999,0.003993
2010-01-08,1144.979980,18.129999,0.002878
2010-01-11,1146.979980,17.549999,0.001745
